In [213]:
workspace = '/content/drive/MyDrive/Courses/CS410/MP4/'

import os
os.path.exists(workspace)

True

In [214]:
 %%capture
 !pip install faiss-cpu ir-measures 

In [215]:
import faiss 
import pickle
import numpy as np
import ir_measures
from ir_measures import *

In [226]:
model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1'
doc_emb_fname = workspace + 'data/documents/documents' + model_name.replace('/','') + '.pkl'
qry_emb_fname = workspace + 'data/queries/queries' + model_name.replace('/','') + '.pkl'

os.path.exists(doc_emb_fname), os.path.exists(qry_emb_fname)

(True, True)

In [227]:
with open(doc_emb_fname, 'rb') as f:
  doc_embeddings = pickle.load(f)

len(doc_embeddings)

7014

In [228]:
with open(qry_emb_fname, 'rb') as f:
  qry_embeddings = pickle.load(f)

len(qry_embeddings)

2234

In [229]:
d = 768 # The dimensionality of out embeddings
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
print(index.ntotal)

index.add(np.squeeze(np.array(doc_embeddings)))
print("Our Index now has {} documents".format(index.ntotal))

True
0
Our Index now has 7014 documents


In [230]:
# k = len(doc_embeddings)

top_n = 20 # keep top n in ranking
k = top_n

distances, indexes = index.search(np.squeeze(np.array(qry_embeddings)), k) 
indexes.shape

(2234, 20)

In [231]:
indexes[0]

array([6556, 1920,  635, 4671,    3, 4259, 2488, 1918, 2075,   24, 3838,
       7011,   10,   27, 1786, 2393,    0, 3258,  612, 1122])

In [232]:
rank_file = workspace + 'MP4.1-bi-encoder-candidate-ranking.trec'

#query_id starts at 1, document_id starts at 0???, and rank starts at 1 
with open(rank_file, 'w') as w:
  for i in range(0, indexes.shape[0]): 
      for j in range(0, top_n):
        w.write("{}\t0\t{}\t{}\t{}\tcandidate-run\n".format(i+1, indexes[i][j], j+1, -distances[i][j]))  #QUERY_ID\t0\tDOCUMENT_ID\tRANK\tSCORE\trun_id

In [233]:
qrels = ir_measures.read_trec_qrels(workspace + 'data/qrels.trec')
run = ir_measures.read_trec_run(rank_file)

In [234]:
names = [NDCG@3, NDCG@5, NDCG@10, NDCG@20, R@5, R@10, R@100, R@1000]
evals = ir_measures.calc_aggregate(names, qrels, run)

In [235]:
for name in names:
  print("{:10s} = {:.3f}".format(str(name), evals[name]))

nDCG@3     = 0.343
nDCG@5     = 0.323
nDCG@10    = 0.305
nDCG@20    = 0.329
R@5        = 0.147
R@10       = 0.227
R@100      = 0.339
R@1000     = 0.339
